In [1]:
using LinearAlgebra

In [8]:
function classic_(A, B, C)
    len = size(A,1)
    for i in 1:len
       for j in 1:len
           for k in 1:len
                C[i, j] += (A[i, k] * B[k, j])
            end
        end
    end
    return C
end

classic_ (generic function with 1 method)

In [25]:
function multiply_rec(A,B,C)
    if size(A,1) == 2
        C = classic_(A, B, C)
        return C
    end
    len = (size(A,1)÷2)
    idx = [ [1:len,1:len],
            [1:len,len+1:len*2],
            [len+1:len*2,1:len],
            [len+1:len*2,len+1:len*2]
          ]

    C[idx[1][1], idx[1][2]] .= multiply_rec(A[idx[1][1], idx[1][2]], 
                 B[idx[1][1], idx[1][2]],
                 C[idx[1][1], idx[1][2]])

    C[idx[1][1], idx[1][2]] .= multiply_rec(A[idx[2][1], idx[2][2]], 
                 B[idx[3][1], idx[3][2]],
                 C[idx[1][1], idx[1][2]])

    C[idx[2][1], idx[2][2]] .= multiply_rec(A[idx[1][1], idx[1][2]], 
                 B[idx[2][1], idx[2][2]],
                C[idx[2][1], idx[2][2]])
    C[idx[2][1], idx[2][2]] .= multiply_rec(A[idx[2][1], idx[2][2]], 
                 B[idx[4][1], idx[4][2]],
                 C[idx[2][1], idx[2][2]])
    
    C[idx[3][1], idx[3][2]] .= multiply_rec(A[idx[3][1], idx[3][2]], 
                 B[idx[1][1], idx[1][2]],
                 C[idx[3][1], idx[3][2]])
    C[idx[3][1], idx[3][2]] .= multiply_rec(A[idx[4][1], idx[4][2]], 
                 B[idx[3][1], idx[3][2]],
                 C[idx[3][1], idx[3][2]])
    
    C[idx[4][1], idx[4][2]] .= multiply_rec(A[idx[3][1], idx[3][2]], 
                 B[idx[2][1], idx[2][2]],
                 C[idx[4][1], idx[4][2]])
    C[idx[4][1], idx[4][2]] .= multiply_rec(A[idx[4][1], idx[4][2]], 
                 B[idx[4][1], idx[4][2]],
                 C[idx[4][1], idx[4][2]])
    return C
end

multiply_rec (generic function with 1 method)

In [129]:
function inv2x2(A)
    detA = A[1, 1]*A[2, 2] - A[1, 2]*A[2, 1]
    invA = [
        A[2, 2] -A[1, 2]
        -A[2, 1] A[1, 1]
    ]
    return (1/detA)*invA
end

function inv(A)
    len    = (size(A,1)÷2)
    invA11 = 1*zeros(len, len)
    B      = 1*zeros(size(A,1), size(A,1))
    S22    = 1*zeros(len, len)
    invS22 = 1*zeros(len, len)
    
    if size(A, 1) == 2
        invA = inv2x2(A)
        return invA
    end
    
    idx = [ [1:len,1:len],
            [1:len,len+1:len*2],
            [len+1:len*2,1:len],
            [len+1:len*2,len+1:len*2]
          ]

    invA11= inv(A[idx[1][1], idx[1][2]])
    
    S22 = A[idx[4][1], idx[4][2]] - 
            (multiply_rec(
                    multiply_rec(A[idx[3][1], idx[3][2]], invA11), A[idx[2][1], idx[2][2]]))
    invS22 = inv(S22)
    Id = 1.0*Matrix(I, len, len)
    
    B[idx[1][1], idx[1][2]] = multiply_rec(invA11, 
    (Id + multiply_rec(multiply_rec(multiply_rec(A[idx[2][1],idx[2][2]], invS22), A[idx[3][1], idx[3][2]]), invA11)))
    B[idx[2][1], idx[2][2]] = multiply_rec(multiply_rec(-invA11, A[idx[2][1], idx[2][2]]), invS22)
    B[idx[3][1], idx[3][2]] = multiply_rec(multiply_rec(-invS22, A[idx[3][1], idx[3][2]]), invA11)
    B[idx[4][1], idx[4][2]] = invS22
    
    return B
end

inv (generic function with 2 methods)

In [130]:
b = [
    3.0 4.0 2.0 4.0
    1.0 4.0 3.0 4.0
    3.0 1.0 3.0 2.0
    3.0 4.0 3.0 4.0
]

invB = inv(b)

4×4 Matrix{Float64}:
  0.0   -0.5     0.0   0.5
 -1.5   -0.75   -1.0   2.75
 -1.0    0.0    -0.0   1.0
  2.25   1.125   1.0  -3.625

In [125]:
b = [
    3.0 4.0 
    1.0 4.0 
]
println(b[1:2, 1:2])
invB = inv(b)

[3.0 4.0; 1.0 4.0]


2×2 Matrix{Float64}:
  0.5    -0.5
 -0.125   0.375

In [7]:
a = [
    1 2 1 2
    1 2 1 2
    1 2 1 2
    1 2 1 2
]

b = [
    3 4 3 4
    3 4 3 4
    3 4 3 4
    3 4 3 4
]

c = [
    0 0 0 0
    0 0 0 0
    0 0 0 0
    0 0 0 0
]

c = multiply_rec(a, b)
print(c)

[9.0 12.0 9.0 12.0; 9.0 12.0 9.0 12.0; 9.0 12.0 9.0 12.0; 9.0 12.0 9.0 12.0]

In [23]:
a = rand(8, 8)
b = rand(8, 8)
c = zeros(8, 8)
multiply_rec(a, b, c)

8×8 Matrix{Float64}:
 1.73019   2.39879  1.3719   2.14658  2.04498  2.19401  2.44571  1.90419
 1.40316   2.13915  1.36556  1.63464  1.65294  1.85017  2.24554  1.70807
 2.12475   3.21503  1.11675  2.4686   2.47648  2.81604  2.99111  2.53573
 1.1187    2.64629  1.98468  1.11402  1.89851  2.0483   3.13119  1.65501
 1.50045   2.49037  1.27578  1.05489  1.93098  1.96441  2.82401  1.82511
 0.711509  1.82084  1.28939  1.67275  1.85281  1.52796  1.92185  1.46344
 0.770028  1.5182   1.16382  1.68534  1.4015   1.44134  1.55155  1.17055
 1.50353   3.06731  2.11574  2.83961  3.33885  2.47593  3.45483  2.78922

In [24]:
a*b

8×8 Matrix{Float64}:
 1.73019   2.39879  1.3719   2.14658  2.04498  2.19401  2.44571  1.90419
 1.40316   2.13915  1.36556  1.63464  1.65294  1.85017  2.24554  1.70807
 2.12475   3.21503  1.11675  2.4686   2.47648  2.81604  2.99111  2.53573
 1.1187    2.64629  1.98468  1.11402  1.89851  2.0483   3.13119  1.65501
 1.50045   2.49037  1.27578  1.05489  1.93098  1.96441  2.82401  1.82511
 0.711509  1.82084  1.28939  1.67275  1.85281  1.52796  1.92185  1.46344
 0.770028  1.5182   1.16382  1.68534  1.4015   1.44134  1.55155  1.17055
 1.50353   3.06731  2.11574  2.83961  3.33885  2.47593  3.45483  2.78922

In [11]:
a = rand(4, 4)
b = rand(4, 4)
c = zeros(4, 4)
multiply_rec(a, b, c)

4×4 Matrix{Float64}:
 0.857214  0.944818  0.780652  0.4998
 1.38742   1.19131   0.697611  0.688935
 1.33745   1.32616   0.82311   0.667173
 1.8459    1.67835   1.07516   0.937895

In [12]:
a*b

4×4 Matrix{Float64}:
 0.857214  0.944818  0.780652  0.4998
 1.38742   1.19131   0.697611  0.688935
 1.33745   1.32616   0.82311   0.667173
 1.8459    1.67835   1.07516   0.937895

In [9]:
a = [
    1 2 1 2
    1 2 1 2
    1 2 1 2
    1 2 1 2
]

b = [
    3 4 3 4
    3 4 3 4
    3 4 3 4
    3 4 3 4
]

c = [
    0 0 0 0
    0 0 0 0
    0 0 0 0
    0 0 0 0
]

c = multiply_rec(a, b, c)
print(c)

[18 24 18 24; 18 24 18 24; 18 24 18 24; 18 24 18 24]

In [6]:
a*b

4×4 Matrix{Int64}:
 18  24  18  24
 18  24  18  24
 18  24  18  24
 18  24  18  24